<a href="https://colab.research.google.com/github/Hab-eeb/Flower_color_detection/blob/main/color_det_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>